In [46]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import prince

In [47]:
df = pd.read_csv('data/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2018.csv')
df.head()

C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,Capital/Adirond,Columbia,1001000.0,146.0,Columbia Memorial Hospital,50 to 69,125,F,White,Not Span/Hispanic,...,Major,Major,Medical,"Managed Care, Unspecified",Self-Pay,NaN,NaN,True,17591.41,3570.06
1,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,Extreme,Extreme,Surgical,Medicare,Medicaid,NaN,NaN,True,106190.21,78913.24
2,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,False,5600.00,4161.53
3,New York City,Kings,7001016.0,1301.0,Kings County Hospital Center,18 to 29,112,M,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,Medicaid,NaN,NaN,True,15965.54,11341.30
4,New York City,Manhattan,7002054.0,1464.0,New York-Presbyterian Hospital - Columbia Pres...,0 to 17,104,M,Multi-racial,Not Span/Hispanic,...,Major,Moderate,Medical,Medicaid,Medicaid,Self-Pay,NaN,True,36846.83,10240.02


In [48]:
# Cleaning steps
# Drop any columns with > 30% null values
df_cleaned = df.drop(['Birth Weight', 'Payment Typology 3', 'Payment Typology 2', 'CCSR Procedure Code', 'CCSR Procedure Description'], axis=1)
df_cleaned.dropna(inplace=True)
df_cleaned = df_cleaned[df_cleaned['Length of Stay'] != '120 +']
df_cleaned.head()

,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Emergency Department Indicator,Total Charges,Total Costs
0,Capital/Adirond,Columbia,1001000.0,146.0,Columbia Memorial Hospital,50 to 69,125,F,White,Not Span/Hispanic,...,1.0,DISEASES AND DISORDERS OF THE NERVOUS SYSTEM,3.0,Major,Major,Medical,"Managed Care, Unspecified",True,17591.41,3570.06
1,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,4.0,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,4.0,Extreme,Extreme,Surgical,Medicare,True,106190.21,78913.24
2,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,19.0,MENTAL DISEASES AND DISORDERS,2.0,Moderate,Minor,Medical,Medicaid,False,5600.00,4161.53
3,New York City,Kings,7001016.0,1301.0,Kings County Hospital Center,18 to 29,112,M,Black/African American,Not Span/Hispanic,...,5.0,DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM,2.0,Moderate,Minor,Medical,Medicaid,True,15965.54,11341.30
4,New York City,Manhattan,7002054.0,1464.0,New York-Presbyterian Hospital - Columbia Pres...,0 to 17,104,M,Multi-racial,Not Span/Hispanic,...,4.0,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,3.0,Major,Moderate,Medical,Medicaid,True,36846.83,10240.02


In [49]:
#Predict Length of Stay w/chest pain subgroup
print(len(df_cleaned[df_cleaned['APR DRG Description'].str.startswith('HEART')]))
print(df_cleaned[df_cleaned['APR DRG Description'].str.startswith('HEART')][['Age Group', 'Gender', 'APR DRG Code', 'APR DRG Description']].head(100))
# y = df_cleaned[['Length of Stay']]
# X = df_cleaned[['Age Group', 'Type of Admission', 'Patient Disposition', 'APR Severity of Illness Code', 'APR Severity of Illness Description', 'APR Risk of Mortality', 'Emergency Department Indicator']]

# print(y.columns)

64929
        Age Group Gender  APR DRG Code APR DRG Description
47    70 or Older      M         194.0       HEART FAILURE
83       50 to 69      M         194.0       HEART FAILURE
134      30 to 49      F         194.0       HEART FAILURE
141   70 or Older      M         194.0       HEART FAILURE
220   70 or Older      F         194.0       HEART FAILURE
...           ...    ...           ...                 ...
3840  70 or Older      M         194.0       HEART FAILURE
3912     50 to 69      M         194.0       HEART FAILURE
3932     50 to 69      M         194.0       HEART FAILURE
3969  70 or Older      M         194.0       HEART FAILURE
4003  70 or Older      M         194.0       HEART FAILURE

[100 rows x 4 columns]


In [50]:
heart_failure_patients_df = df_cleaned[df_cleaned['APR DRG Description'].str.startswith('HEART')]

y = heart_failure_patients_df[['Length of Stay']]
X = heart_failure_patients_df.loc[:, ~heart_failure_patients_df.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]

In [51]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor

X_train_all_features, X_test_all_features, y_train_all_features, y_test_all_features = train_test_split(X, y, test_size = 0.25)
# X_train_all_features = df_cleaned.loc[:, ~df_cleaned.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]
categorical_columns = X_train_all_features.select_dtypes(exclude='number')

for column in categorical_columns:
    values = df_cleaned[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_train_all_features[column].replace(values, encoded_values, inplace=True)
    
estimator = DecisionTreeRegressor(criterion="squared_error", max_depth=10)
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X_train_all_features, y_train_all_features)

selector.ranking_

array([13, 12,  1,  3,  8,  6,  1, 17, 10,  7,  5,  1, 21,  4, 14,  1, 19,
       11, 18,  1, 15,  9, 20,  2, 16])

In [52]:
X_train_all_features.columns[np.where(selector.ranking_ == 1)]

Index(['Operating Certificate Number', 'Zip Code - 3 digits',
       'Patient Disposition', 'APR DRG Code', 'APR Severity of Illness Code'],
      dtype='object')

In [53]:
for column in categorical_columns:
    values = df_cleaned[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_test_all_features[column].replace(values, encoded_values, inplace=True)


dt = DecisionTreeRegressor(criterion="squared_error", max_depth=10)
dt.fit(X_train_all_features, y_train_all_features)
yhat = dt.predict(X_test_all_features)

mae = metrics.mean_absolute_error(y_test_all_features, yhat)
mse = metrics.mean_squared_error(y_test_all_features, yhat)
r2 = metrics.r2_score(y_test_all_features, yhat)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

The model performance for testing set
--------------------------------------
MAE is 2.99952895350548
MSE is 28.948801849623315
R2 score is 0.21599042757531695


In [54]:
from sklearn.ensemble import RandomForestRegressor


X_train_feature_selection = X_train_all_features[X_train_all_features.columns[np.where(selector.ranking_ == 1)]]
X_test_feature_selection = X_test_all_features[X_test_all_features.columns[np.where(selector.ranking_ == 1)]]

y_train_all_features['Length of Stay'] = y_train_all_features['Length of Stay'].astype('float64')
y_train_all_features['Length of Stay'] = np.log(y_train_all_features['Length of Stay'].values.reshape(-1,1))

#print(y_train_all_features)

y_test_all_features['Length of Stay'] = y_test_all_features['Length of Stay'].astype('float64')
y_test_all_features['Length of Stay'] = np.log(y_test_all_features['Length of Stay'].values.reshape(-1,1))
rf = RandomForestRegressor(max_depth=2, random_state=42)

rf.fit(X_train_feature_selection, y_train_all_features)

preds = rf.predict(X_test_feature_selection)
print(preds)
print(y_test_all_features)
mae = metrics.mean_absolute_error(y_test_all_features, preds)
mse = metrics.mean_squared_error(y_test_all_features, preds)
r2 = metrics.r2_score(y_test_all_features, preds)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

<ipython-input-54-e526d33da480>:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train_feature_selection, y_train_all_features)


[1.56428546 1.56428546 1.07511038 ... 1.86327046 1.56021092 1.56428546]
         Length of Stay
251944         2.079442
1283466        0.693147
1966533        0.693147
1904323        2.302585
65924          0.693147
...                 ...
872174         1.791759
280990         1.945910
1219356        2.079442
420670         1.386294
817618         1.609438

[16233 rows x 1 columns]
The model performance for testing set
--------------------------------------
MAE is 0.5293937262408765
MSE is 0.4605030222668667
R2 score is 0.19833664280082108


In [60]:
#Linear Regressor w/log transform of target variable
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler

y = heart_failure_patients_df[['Length of Stay']]
X = heart_failure_patients_df.loc[:, ~heart_failure_patients_df.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]

X_train_all_features, X_test_all_features, y_train_all_features, y_test_all_features = train_test_split(X, y, test_size = 0.25)
# X_train_all_features = df_cleaned.loc[:, ~df_cleaned.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]
categorical_columns = X_train_all_features.select_dtypes(exclude='number')

for column in categorical_columns:
    values = df_cleaned[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_train_all_features[column].replace(values, encoded_values, inplace=True)
    X_test_all_features[column].replace(values, encoded_values, inplace=True)

lr = LinearRegression()
lr.fit(X_train_all_features, y_train_all_features)
yhat = lr.predict(X_test_all_features)

mae = metrics.mean_absolute_error(y_test_all_features, yhat)
mse = metrics.mean_squared_error(y_test_all_features, yhat)
r2 = metrics.r2_score(y_test_all_features, yhat)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

dt = DecisionTreeRegressor(criterion="squared_error", max_depth=10)
dt.fit(X_train_all_features, y_train_all_features)
yhat = dt.predict(X_test_all_features)

mae = metrics.mean_absolute_error(y_test_all_features, yhat)
mse = metrics.mean_squared_error(y_test_all_features, yhat)
r2 = metrics.r2_score(y_test_all_features, yhat)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))


tt = TransformedTargetRegressor(regressor=LinearRegression(), transformer=MinMaxScaler())
tt.fit(X_train_all_features, y_train_all_features)
tt.score(X_test_all_features, y_test_all_features)

The model performance for testing set
--------------------------------------
MAE is 3.031095445880974
MSE is 26.557275407454707
R2 score is 0.2691571112941529
The model performance for testing set
--------------------------------------
MAE is 3.0036560257397213
MSE is 30.332646818584948
R2 score is 0.16526078511931575


0.26915711129415254

In [64]:
#Linear Regression w/OneHotEncoding
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler

categorical_columns = X_train_all_features.select_dtypes(exclude='number')
print(categorical_columns)
categorical_columns["Emergency Department Indicator"] = categorical_columns['Emergency Department Indicator'].astype('object')
heart_failure_patients_df_encoded = pd.get_dummies(heart_failure_patients_df, columns = categorical_columns)

print(pd.get_dummies(heart_failure_patients_df, columns = categorical_columns))
y = heart_failure_patients_df[['Length of Stay']]
X = heart_failure_patients_df.loc[:, ~heart_failure_patients_df.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]

X_train_all_features, X_test_all_features, y_train_all_features, y_test_all_features = train_test_split(X, y, test_size = 0.25)
# X_train_all_features = df_cleaned.loc[:, ~df_cleaned.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]
categorical_columns = X_train_all_features.select_dtypes(exclude='number')

'''for column in categorical_columns:
    values = df_cleaned[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_train_all_features[column].replace(values, encoded_values, inplace=True)
    X_test_all_features[column].replace(values, encoded_values, inplace=True)'''

lr = LinearRegression()
lr.fit(X_train_all_features, y_train_all_features)
yhat = lr.predict(X_test_all_features)

mae = metrics.mean_absolute_error(y_test_all_features, yhat)
mse = metrics.mean_squared_error(y_test_all_features, yhat)
r2 = metrics.r2_score(y_test_all_features, yhat)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

        Hospital Service Area Hospital County  \
1019893         New York City           Kings   
298699             Western NY            Erie   
855315          New York City           Kings   
291380            Long Island          Nassau   
1108096       Capital/Adirond          Albany   
...                       ...             ...   
1934466         Hudson Valley     Westchester   
1293573         New York City           Kings   
204680          Southern Tier          Broome   
2015669       Capital/Adirond      Rensselaer   
801925          New York City       Manhattan   

                                             Facility Name    Age Group  \
1019893         Brooklyn Hospital Center - Downtown Campus  70 or Older   
298699                           Mercy Hospital of Buffalo     50 to 69   
855315   New York - Presbyterian Brooklyn Methodist Hos...  70 or Older   
291380                                 St. Joseph Hospital     50 to 69   
1108096                             

<ipython-input-64-2d5bc68d97b8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_columns["Emergency Department Indicator"] = categorical_columns['Emergency Department Indicator'].astype('object')


ValueError: Boolean array expected for the condition, not object

In [65]:
df = pd.get_dummies(heart_failure_patients_df['APR Risk of Mortality'], columns = ['APR Risk of Mortality'])
df.head()

,Extreme,Major,Minor,Moderate
47,0,1,0,0
83,0,0,0,1
134,0,0,0,1
141,0,0,0,1
220,0,0,0,1
